<a href="https://colab.research.google.com/github/varvaraptl/Our_Poject/blob/My_branch/easy_ad_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyyaml==5.1
# Torch version 1.9.0
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Instruction https://detectron2.readthedocs.io/tutorials/install.html 
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
##### Restart a  Runtime ####

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 4.8 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=404403d8080f81cbb6e9547532a5bd5f72cde31911180d38c57986caed17440e
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/

In [ ]:
# Check versions
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9") 

1.9.0+cu102 True


In [ ]:
import os
from os.path import isfile, join
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from PIL import Image
import imutils

In [ ]:
# Install logger for detectron
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Imports
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# Use default config
cfg = get_cfg()

# Download config for mask_rcnn_R_101_C4_3x model
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml"))

# Installing threshlod for detection
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3

# Download model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml")

# Create object predictor
predictor = DefaultPredictor(cfg)

model_final_a2914c.pkl: 220MB [01:02, 3.50MB/s]                           


In [ ]:
def getFrame(sec, path_shots, vidcap, count):
    '''function gets shot from video and write it down to folder 'path_shots.
    use this function with storyboard function'''

    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames, image = vidcap.read()
    image = cv2.rotate(image, cv2.ROTATE_180)
    if hasFrames:
      # saves frame as JPG file
        cv2.imwrite(path_shots + "/shot"+str(count)+".jpg", image)     
    return hasFrames

In [ ]:
def storyboard(video, path_shots, frameRate = 0.05):
    '''function cuts video with presetted frameRate.
    frameRate = 0.05 (default) means that it will capture image in each 0.05 second'''

    # reads a video
    vidcap = cv2.VideoCapture(video)

    # makes a path
    if not os.path.exists(path_shots):
        os.mkdir(path_shots)

    # storyboards a video
    sec = 0
    count=1
    success = getFrame(sec, path_shots, vidcap, count)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec, path_shots, vidcap, count)

    # makes list of shots names
    files = [f for f in os.listdir(path_shots) if isfile(join(path_shots, f))]

    # sorts shots
    files = sorted(files, key = lambda x: int(x.split('.')[0][5:]))

    return files, vidcap

In [ ]:
def find_contour(mask):
    '''function finds max contour for mask and returns it's box and angle'''
    
    # initializes a mask of an object
    img = (255*np.array(mask)).astype(np.uint8)

    # smoothes a mask
    img = cv2.GaussianBlur(img, (5, 5), 0)
  
    # determines the form of an object we are looking for
    thrs = 32
    retval, bin = cv2.threshold(img, thrs, 255, cv2.THRESH_BINARY)

    #finding contours
    contours, hierarchy = cv2.findContours(bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) ## cv2.RETR_EXTERNAL 

    max=0

    # finds sel contour
    sel_contour=None
    if len(contours) != 0:
        sel_contour = sorted(contours, key = lambda x: x.shape[0])[-1]

        # finds the convex hull of a point set.
        hull = cv2.convexHull(sel_contour)

        # finds min rectangle area
        rect = cv2.minAreaRect(hull) # center (x, y), (w, h), angle

        # finds corner points of rectangle
        box = cv2.boxPoints(rect) # 4 точки 
        box = box.astype(int)

        #fings angle of rotation
        angle = cv2.minAreaRect(sel_contour)[-1]
    else:
        box = []
        angle = 0

    return box, angle

In [ ]:
def rotate_dots(dots, direction):
    '''function finds angle for logo rotation in radians'''

    phi = np.radians(direction)
    
    m_rot = np.array([[np.cos(phi),-np.sin(phi)],
                     [np.sin(phi), np.cos(phi)]])
    
    return dots @ m_rot

In [ ]:
def rectangle_corners(proj_x, proj_y, proj_size_x, proj_size_y, proj_bias_x, proj_bias_y, direction) -> np.array:
    '''Function counts corners coordinates of rotated rectangle, returns np.array'''
    
    # finds rectangle corners without rotation and shifting
    rect_corners_normal = np.array([[- proj_size_x / 2 - proj_bias_x, -proj_size_y / 2 - proj_bias_y],
                                    [+ proj_size_x / 2 - proj_bias_x, -proj_size_y / 2 - proj_bias_y],
                                    [+ proj_size_x / 2 - proj_bias_x, proj_size_y / 2 - proj_bias_y],
                                    [- proj_size_x / 2 - proj_bias_x, proj_size_y / 2 - proj_bias_y]])
    
    # rotates rectangle on direction angle
    rect_corners_rotated = rotate_dots(rect_corners_normal, direction)
        
    # returns np.array with shift
    return rect_corners_rotated + np.array([proj_x,proj_y])

In [ ]:
def make_transparent_background(img):
    '''function makes image background transparent'''

    img = img.astype(np.uint8)
    img = Image.fromarray(img)

    rgba = img.convert("RGBA")
    datas = rgba.getdata()

    newData = []
    for item in datas:
    # finds black colour by its RGB value
        if item[0] == 0 and item[1] == 0 and item[2] == 0:
            # stores a transparent value when we find a black colour
            newData.append((255, 255, 255, 0))
        else:
            # other colours remain unchanged
            newData.append(item)

    rgba.putdata(newData)
    return rgba

In [ ]:
def frame_photo(img, n):
    '''functions adds sides padding to image using the extreme pixels'''
  
    new_img = np.array([])
    color_num = len(img[0][0])
    for i in range(len(img)):
        pix_first = img[i][0]
        pix_last = img[i][-1]
        line = img[i]
        for j in range(n):
            line = np.append(line, pix_last)
            line = np.insert(line, 0, pix_first)
        line = line.reshape(len(img[i]) + (n * 2), color_num)
        new_img = np.append(new_img, line)
    new_img = new_img.reshape(len(img), len(line), color_num)  
    
    return new_img

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    '''fuctions counts intersection over union of 2 bounding boxes'''
    
    # determines the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # computes the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
    # computes the area of both the prediction and ground-truth
    
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # computes the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    # returns the intersection over union value
    return iou

In [ ]:
def find_masks(outputs):
    '''function finds masks of detected objects'''

    # initializes list of objects to put a logo on it
    # here it takes tv and cellphone
    tv_cell_list = [ind for ind, val in enumerate(outputs['instances'].pred_classes.tolist()) if val == 62 or val == 67]

    # initializes list of objects to exclude them from iou comparison
    other_objects_list = [ind for ind, val in enumerate(outputs['instances'].pred_classes.tolist()) if val != 62 and val != 63 and val != 67 and val != 0]

    # findes objects instances
    tv_cell_instances = outputs['instances'].__getitem__(tv_list)
    other_objects_instances = outputs['instances'].__getitem__(other_objects_list)

    # finds objects bounding boxes
    tv_cell_boxes = tv_instances.pred_boxes
    other_boxes = other_objects_instances.pred_boxes

    # finds objects scores
    tv_cell_scores = tv_instances.scores
    other_scores = other_objects_instances.scores

    # tunes iou and scores threshold
    tv_list_for_del = []

    for ind_tv, tv_box in enumerate(tv__cell_boxes):
    for ind_other, other_box in enumerate(other_boxes):
        iou = bb_intersection_over_union(tv_box, other_box)
        # with next line we can find masks not only for tvs, but for laptops' screens as well
        if iou > 0.3 and other_scores[ind_other] > 0.5 or tv_cell_scores[ind_tv]<0.1:
            tv_list_for_del.append(ind_tv)

    tv_list_for_del = sorted(set(tv_list_for_del))
    tv_list_for_del.reverse()
    # deletes tvs, that as also laptops, so we will not have double masks
    for ind in tv_list_for_del:
    del tv_cell_list[ind]

    tv_cell_masks = outputs['instances'].pred_masks[tv_cell_list]

    # returns masks of screens
    return tv_cell_masks

In [ ]:
def put_on_mask(masks, im, adv):
    '''function agregates fucnctions: 'find_contour',  'frame_photo', 'rectangle_corners', 'make_transparent_background' 
    and pastes logo on an object mask place with rotation'''

    for ind, mask in enumerate(masks):
    mask = mask.to('cpu').numpy()

    #  finds corner coordinates and angle of a screen
    box, angle  = find_contour(mask)
    if len(box) != 0:
        box = sorted(box, key = lambda x:x[0])
        left_high_dot, left_low_dot, right_high_dot, right_low_dot = box
        if left_high_dot[1] > left_low_dot[1]:
            left_high_dot, left_low_dot = left_low_dot, left_high_dot
        if right_high_dot[1] > right_low_dot[1]:
            right_high_dot, righ_low_dot = right_low_dot, right_high_dot

        # counts height and width
        h, w = abs(left_high_dot[1] - left_low_dot[1]), abs(left_high_dot[0] - right_high_dot[0])

        # resizes logo to a square with height size side
        logo = adv.resize((h, h))

        # adds sides padding
        if w > h:
            pixels = int((w-h)/2)
        else:
            pixels = 0

        logo = np.array(logo)
        if pixels != 0:
            logo = frame_photo(logo, pixels)

        # finds a top left corner of a logo so it perfectly fits top left corner of a screen for clock-wise rotation
        if angle < 45:
            left_shift_rotation = rectangle_corners(0, 0, w, h, 0, 0, angle)[1][0] + rectangle_corners(0, 0, w, h, 0, 0, angle)[-1][0]
            left_high_dot[0] = left_high_dot[0] - left_shift_rotation

        # finds a top left corner of a logo so it perfectly fits top left corner of a screen for counterclock-wise rotation
        else:
            angle = angle - 90
            down_shift_rotation =  rectangle_corners(0, 0, w, h, 0, 0, angle)[-2][1] - rectangle_corners(0, 0, w, h, 0, 0, angle)[-1][1]
            left_high_dot[1] = left_high_dot[1] - down_shift_rotation

        # rotates logo
        logo = imutils.rotate_bound(logo, angle)

        # makes logo backgroud transparent after rotation
        logo = make_transparent_background(logo)

        print(logo)

        im.paste(
            logo,
            tuple(left_high_dot), logo)

    return im

In [ ]:
def create_video(path_video_out, frame_out, files, vidcap, fps):
    '''functions writes down a video out of a shots with logo'''
    out = cv2.VideoWriter(path_video_out,cv2.VideoWriter.fourcc(*'XVID'), fps, (int(vidcap.get(3)),int(vidcap.get(4))))
    for i in range(len(files)):

    # writes to a image array
    img = cv2.imread(f'{frame_out}/image{i}.jpg')
    out.write(img)

    out.release()

In [ ]:
def ads_video(video, ads_photo, path_video_out, path_shots, frame_out, frameRate):
    '''main function that gets a video, logo, paths for shots before and after placing a logo into the screens and path for new video'''

    files, vidcap = storyboard(video, path_shots, frameRate)

    # converts cv2 to PIL
    with Image.open(ads_photo).convert('RGBA') as adv:
        adv.load()

    adv.save(f'test.png')

    for i in tqdm(range(len(files))):

        filename= f'{path_shots}/{files[i]}'

        #reads each files
        im = filename
        
        with Image.open(im) as im:
            im.load()

        size = im.size

        # converts PIL to cv2
        img = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)

        # applies MASK R-CNN model
        outputs = predictor(img)

        # finds masks
        masks = find_masks(outputs)

        # pastes logo into a mask
        im = put_on_mask(masks, im, adv)
        
        if not os.path.exists(frame_out):
            os.mkdir(frame_out)

        im.save(f'{frame_out}/image{i}.jpg')
        
    # creates a new video
    fps = int(1/frameRate)

    create_video(path_video_out, frame_out, files, vidcap, fps)

In [ ]:
video = '/content/drive/MyDrive/IMG_8115.MOV'
adv = "/content/drive/MyDrive/elbrus.png"
path_video_out = '/content/drive/MyDrive/video_final!!!.MOV'
path_shots = '/content/drive/MyDrive/shots100'
frame_out = '/content/drive/MyDrive/images100'

ads_video(video, adv, path_video_out, path_shots, frame_out, 1/50)

  0%|          | 0/3159 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  1%|▏         | 40/3159 [00:25<35:07,  1.48it/s]

<PIL.Image.Image image mode=RGBA size=30x30 at 0x7F52C0BC2F10>
<PIL.Image.Image image mode=RGBA size=25x25 at 0x7F52C0BCB510>


  1%|▏         | 41/3159 [00:26<36:14,  1.43it/s]

<PIL.Image.Image image mode=RGBA size=21x21 at 0x7F52C0BC2F50>
<PIL.Image.Image image mode=RGBA size=26x26 at 0x7F52C0BC2C10>
<PIL.Image.Image image mode=RGBA size=32x32 at 0x7F52C0BC2F90>


  1%|▏         | 42/3159 [00:26<35:53,  1.45it/s]

...

<PIL.Image.Image image mode=RGBA size=278x193 at 0x7F5316101690>


100%|█████████▉| 3158/3159 [47:45<00:00,  1.18it/s]

<PIL.Image.Image image mode=RGBA size=277x191 at 0x7F52C0BC2F90>


100%|██████████| 3159/3159 [47:46<00:00,  1.10it/s]

<PIL.Image.Image image mode=RGBA size=278x193 at 0x7F5316113F50>
